# Pre-Requisites

In [ ]:
!jupyter nbextension enable --py widgetsnbextension

In [1]:
from tqdm import tqdm
from bs4 import BeautifulSoup as bs
import requests

In [2]:
nocd = "https://www.treatmyocd.com"
all_articles = "https://www.treatmyocd.com/learn/blog"
blog_categories = set()

resp = requests.get(all_articles)
soup = bs(resp.text, "html.parser")
for tag in soup.find_all(): 
    if tag.name == 'a' and 'href' in tag.attrs:
        if 'blog/category' in tag['href']:
            blog_categories.add(tag['href'])

articles = set()
for cat in tqdm(blog_categories):
    resp = requests.get(nocd + cat)
    soup = bs(resp.text, "html.parser")
    for tag in soup.find_all(): 
        if tag.name == 'a' and 'href' in tag.attrs:
            if 'blog/' in tag['href']:
                articles.add(tag['href'])
                
print(len(articles))

100%|██████████| 3/3 [00:00<00:00,  4.43it/s]

243


In [3]:
blog_text = ['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', None]
paragraph_headings = ['h2', 'h3', 'h4', 'h5', 'h6']

def extract_text_v2(soup, article_name):
    article = soup.select('article')
    article = article[0].select('div[class*="Post"]')
    title = [((article[0].find_all('h1'))[0].text, 'h1', 'title', article_name)]
    body = soup.select('div[class*="Body"]')

    curr_paragraph = None
    lines = []
    for content in body[0].contents:
        if content.name in blog_text:
            try:
                # get text contents
                line = content.getText()
                # ignore \n characters and empty spaces
                if len(line.split()) > 1:
                    # append styles like bold (<em>), links (<a>), etc. to previous ine
                    if content.name not in blog_text:
                        lines[-1] += line
                        continue
                    # grab paragraph name
                    if content.name in paragraph_headings:
                        curr_paragraph = line

                    lines.append((line, content.name, curr_paragraph, article_name))
                    assert len(lines[-1]) == 4, f"Line should only contain 4 objects; {article_name}"
            except Exception:
                lines.append(content)

            

    # body = (body[0].text).split("\n")

    return title + lines

In [4]:
blogs = []
for article in tqdm(articles):
    if article not in blog_categories:
        # print(article)
        resp = requests.get(nocd + article)
        soup = bs(resp.text, "html.parser")
        blog = extract_text_v2(soup, article)
        blogs += blog



100%|██████████| 243/243 [02:05<00:00,  1.93it/s]


In [5]:
import pandas as pd 

df = pd.DataFrame(blogs, columns=['text', 'tag', 'paragraph', 'article'])
df['num_words'] = df['text'].str.split().str.len()
df['num_sentences'] = df['text'].str.split('.').str.len()
df.head(10)

,text,tag,paragraph,article,num_words,num_sentences
0,How to Navigate the Unavoidable Arguments with...,h1,title,/blog/how-to-navigate-the-unavoidable-argument...,11,1
1,"Children not listening, arguments, and disagre...",None,None,/blog/how-to-navigate-the-unavoidable-argument...,71,4
2,How to Avoid Arguments,h2,How to Avoid Arguments,/blog/how-to-navigate-the-unavoidable-argument...,4,1
3,Not everything needs to be a disagreement. If ...,p,How to Avoid Arguments,/blog/how-to-navigate-the-unavoidable-argument...,55,4
4,2. It’s Not Your Fault,p,How to Avoid Arguments,/blog/how-to-navigate-the-unavoidable-argument...,5,2
5,It can be easy to blame yourself or try to fin...,p,How to Avoid Arguments,/blog/how-to-navigate-the-unavoidable-argument...,114,7
6,Although your parenting isn’t to blame for you...,p,How to Avoid Arguments,/blog/how-to-navigate-the-unavoidable-argument...,67,5
7,How to Navigate the Unavoidable Arguments,h2,How to Navigate the Unavoidable Arguments,/blog/how-to-navigate-the-unavoidable-argument...,6,1
8,Active listening is a skill which means it mus...,p,How to Navigate the Unavoidable Arguments,/blog/how-to-navigate-the-unavoidable-argument...,68,5
9,2. Don’t Try to Fix or Cure OCD,p,How to Navigate the Unavoidable Arguments,/blog/how-to-navigate-the-unavoidable-argument...,8,2


In [20]:
df.to_csv('../datasets/blogs_9_20_2022.csv')